In [2]:
print("hello world")

hello world


In [10]:
!pip install python-dotenv google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 713.3/713.3 kB 14.9 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4/4 [google-genai] [google-genai]


In [7]:
import os
from dotenv import load_dotenv
load_dotenv()
api_key = os.getenv("GEMINI_API_KEY")
assert api_key is not None

In [13]:
from google import genai
client = genai.Client()

In [14]:
SAMPLE_TEXT = """
The impact of artificial intelligence on scientific research has been profound. According to a study 
published in Nature (Smith et al., 2023, "AI-Driven Discovery in Materials Science", 
DOI: 10.1038/s41586-023-06221-2), machine learning models have accelerated materials discovery by 50x.

This trend is also reflected in social media discussions. A viral Twitter thread by @AIResearcher 
(https://twitter.com/AIResearcher/status/1234567890) garnered over 50,000 likes discussing how 
GPT-4 is being used in drug discovery pipelines.

The New York Times reported on this phenomenon in their article "The AI Revolution in Labs" 
(https://www.nytimes.com/2023/08/15/science/ai-laboratory-research.html), noting that major 
pharmaceutical companies are investing billions.

A Reddit discussion on r/MachineLearning (https://reddit.com/r/MachineLearning/comments/abc123) 
with 2,400 upvotes debated the reproducibility concerns raised in the original Nature paper.

For a technical deep-dive, the arXiv preprint "Transformer Architectures for Scientific Computing" 
(arXiv:2401.12345) provides implementation details. The official documentation at 
https://docs.example-ai-tool.com/guide has step-by-step tutorials.

Personal blog post by Dr. Jane Chen (https://janechenml.substack.com/p/ai-in-science-2024) 
offers a practitioner's perspective with 15,000 views.

The CDC guidelines (https://www.cdc.gov/ai-health/guidelines.html) now include recommendations 
for AI-assisted diagnostics.
"""

In [15]:
EXTRACTION_PROMPT = """
You are a reference extraction system. Analyze the following text and extract ALL references, citations, and sources mentioned.

For each reference found, provide:
1. raw_text: The exact text/citation as it appears
2. type: One of: academic_journal, academic_preprint, news_media, government_official, social_media, personal_blog, documentation, unknown
3. url: The URL if present (null if not)
4. doi: The DOI if present (null if not)
5. platform: The platform/source (e.g., "Nature", "Twitter", "arXiv", "Reddit", "NYTimes", etc.)
6. engagement_signals: Any mentioned metrics like views, likes, upvotes (null if not mentioned)

Return the results as a JSON array. Be thorough - extract every reference, even implicit ones.

TEXT TO ANALYZE:
---
{text}
---

Return ONLY valid JSON, no markdown formatting or explanation.
"""

In [16]:
import json

def extract_references(text: str) -> list[dict]:
    """Extract references from text using Gemini."""
    
    prompt = EXTRACTION_PROMPT.format(text=text)
    
    response = client.models.generate_content(
        model="gemini-2.0-flash-exp",
        contents=prompt
    )
    
    # Parse the JSON response
    try:
        response_text = response.text.strip()
        # Clean up markdown code blocks if present
        if response_text.startswith("```json"):
            response_text = response_text[7:]
        if response_text.startswith("```"):
            response_text = response_text[3:]
        if response_text.endswith("```"):
            response_text = response_text[:-3]
        
        return json.loads(response_text.strip())
    except json.JSONDecodeError as e:
        print(f"Error parsing JSON: {e}")
        print(f"Raw response: {response.text}")
        return []

In [17]:
references = extract_references(SAMPLE_TEXT)
print(f"Found {len(references)} references:\n")
print(json.dumps(references, indent=2))

Found 8 references:

[
  {
    "raw_text": "Nature (Smith et al., 2023, \"AI-Driven Discovery in Materials Science\", DOI: 10.1038/s41586-023-06221-2)",
    "type": "academic_journal",
    "url": null,
    "doi": "10.1038/s41586-023-06221-2",
    "platform": "Nature",
    "engagement_signals": null
  },
  {
    "raw_text": "@AIResearcher (https://twitter.com/AIResearcher/status/1234567890)",
    "type": "social_media",
    "url": "https://twitter.com/AIResearcher/status/1234567890",
    "doi": null,
    "platform": "Twitter",
    "engagement_signals": "50,000 likes"
  },
  {
    "raw_text": "The New York Times reported on this phenomenon in their article \"The AI Revolution in Labs\" (https://www.nytimes.com/2023/08/15/science/ai-laboratory-research.html)",
    "type": "news_media",
    "url": "https://www.nytimes.com/2023/08/15/science/ai-laboratory-research.html",
    "doi": null,
    "platform": "NYTimes",
    "engagement_signals": null
  },
  {
    "raw_text": "Reddit discussion on